In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()
DATA_PATH = os.environ["DATA_PATH"]
RESULTS_PATH = os.environ["RESULTS_PATH"]
RITS_API_KEY = os.environ["RITS_API_KEY"]

## Coverage Evaluator Test

In [5]:
from src.fact_reasoner.coverage_evaluator import CoverageEvaluator

MODEL = "gpt-oss-20b"
# MODEL = "llama-3.3-70b-instruct"
# MODEL = "Qwen2.5-72B-Instruct"

question = "How is the Loretto Chapel staircase held together?"
contexts = {
    "context1_self_contained": "The Loretto Chapel staircase lacks the newel or central pole usually used to support and stabilize a spiral staircase, and therefore the means of supporting the weight is not obvious. However, the staircase is supported by its stringers just like a conventional (straight) staircase, although in this case each stringer is twisted into a helix. Observers have also noted that the inside stringer has such a tight radius that it is able to function similarly to a straight center support. According to an analysis by a professional carpenter in Mysterious New Mexico, the assembly of the stringers from overlapping segments joined by wood glue[inconsistent] creates a laminate that is actually stronger than the wood alone. Additionally, the use of wooden pegs rather than nails prevents degradation of the joints due to compression set as the wood swells against the nails due to changes in humidity or temperature",
    "context_2_self_contained": 'Loretto Chapel is best known for its helix-shaped staircase (nicknamed "Miraculous Stair"), which rises 20 feet (6.1 m) to the choir loft while making two full turns, all without the support of a newel or central pole. The staircase is built mostly out of wood and is held together by wooden pegs, with no glue, nails or other hardware used. The inner stringer consists of seven wooden segments joined together with pegs, while the longer outer stringer has nine segments. The exact wood used to build the staircase has been confirmed to be a type of spruce which is not native to New Mexico and scientifically not identified anywhere else in the world.',
}
answer = "The Loretto Chapel staircase is held together by wooden pegs, with no glue, nails or other hardware used."

coverage_evaluator = CoverageEvaluator(model=MODEL, version="base")
results = coverage_evaluator.run(question, contexts, answer)

[CoverageEvaluator] Using LLM on RITS: gpt-oss-20b
[CoverageEvaluator] Prompt created: 1
**Final output:**

[Covered statements]
- The staircase is held together by wooden pegs, with no glue, nails or other hardware used. [2]

[Uncovered statements]
- The staircase lacks a newel or central pole. [1]
- The staircase is supported by its stringers like a conventional staircase, with each stringer twisted into a helix. [1]
- The inside stringer has a tight radius that functions similarly to a straight center support. [1]
- The assembly of the stringers from overlapping segments joined by wood glue creates a laminate stronger than the wood alone. [1]
- The use of wooden pegs rather than nails prevents degradation of the joints due to compression set as the wood swells against the nails due to changes in humidity or temperature. [1]
- The staircase is built mostly out of wood. [2]
- The inner stringer consists of seven wooden segments joined together with pegs. [2]
- The outer stringer has n

In [6]:
results

([{'statement': 'The staircase is held together by wooden pegs, with no glue, nails or other hardware used.',
   'is_covered': True,
   'context_ids': ['context_2_self_contained']}],
 [{'statement': 'The staircase lacks a newel or central pole.',
   'is_covered': False,
   'context_ids': ['context1_self_contained']},
  {'statement': 'The staircase is supported by its stringers like a conventional staircase, with each stringer twisted into a helix.',
   'is_covered': False,
   'context_ids': ['context1_self_contained']},
  {'statement': 'The inside stringer has a tight radius that functions similarly to a straight center support.',
   'is_covered': False,
   'context_ids': ['context1_self_contained']},
  {'statement': 'The assembly of the stringers from overlapping segments joined by wood glue creates a laminate stronger than the wood alone.',
   'is_covered': False,
   'context_ids': ['context1_self_contained']},
  {'statement': 'The use of wooden pegs rather than nails prevents degrad

## Answer Comparison Test

In [2]:
MODEL = "gpt-oss-120b"
# MODEL = "llama-3.3-70b-instruct"
# MODEL = "Qwen2.5-72B-Instruct"
USE_TOOLS = False

In [3]:
from src.fact_reasoner.qag_processor import QagProcessor
from src.fact_reasoner.qag_utils import (
    compare_quantities_with_units_definition,
)
from src.fact_reasoner.query_context_summarizer import QueryContextSummarizer

query_context_summarizer = QueryContextSummarizer(
    model=MODEL,
)
qag_processor = QagProcessor(
    model=MODEL,
    comparison_tools=[
        compare_quantities_with_units_definition,
    ]
    if USE_TOOLS
    else [],
)

[QueryContextSummarizer] Using LLM on RITS: gpt-oss-120b
[QagProcessor] Using LLM on RITS: gpt-oss-120b
[QagProcessor] Registered tools for comparison: 


In [4]:
from src.fact_reasoner.qag_processor import Answer, QuestionData

test_data = [
    QuestionData(
        question="What is the maximum firing range of the Blaster (flamethrower)?",
        relevance=5.0,
        answers=[
            Answer("10 meters", confidence=5.0, source_id="c1", source_type="context"),
            Answer("5-metre", confidence=5.0, source_id="c1", source_type="context"),
            Answer("2 meters", confidence=5.0, source_id="c3", source_type="context"),
            Answer("6 ft 7 in", confidence=5.0, source_id="c4", source_type="context"),
        ],
    ),
    QuestionData(
        question="How many monks know the secret recipe for Chartreuse?",
        relevance=5.0,
        answers=[
            Answer("3 monks", confidence=5.0, source_id="c1", source_type="context"),
            Answer("two", confidence=5.0, source_id="c1", source_type="context"),
            Answer(
                "at least two monks",
                confidence=5.0,
                source_id="c2",
                source_type="context",
            ),
            Answer("unknown", confidence=5.0, source_id="c1", source_type="context"),
        ],
    ),
    QuestionData(
        question="Which people took part in the OpenTerella project?",
        relevance=5.0,
        answers=[
            Answer(
                "Jacob Bailey and Gustav Anderson",
                confidence=5.0,
                source_id="c1",
                source_type="context",
            ),
            Answer(
                "Jacob Bailey", confidence=5.0, source_id="c2", source_type="context"
            ),
            Answer(
                "Lucas Hemsworth", confidence=5.0, source_id="c3", source_type="context"
            ),
        ],
    ),
    QuestionData(
        question="How effectively can machine learning techniques, integrating SAR, and optical data, differentiate oil palm plantations from natural forests?",
        relevance=5.0,
        answers=[
            Answer("84% OA", confidence=5.0, source_id="c1", source_type="context"),
            Answer("92.1% OA", confidence=5.0, source_id="c2", source_type="context"),
            Answer(
                "effectively", confidence=5.0, source_id="c3", source_type="context"
            ),
            Answer(
                "very effectively",
                confidence=5.0,
                source_id="c4",
                source_type="context",
            ),
            Answer(
                "with high accuracy",
                confidence=5.0,
                source_id="c5",
                source_type="context",
            ),
            Answer(
                "59% accuracy", confidence=5.0, source_id="c6", source_type="context"
            ),
        ],
    ),
    QuestionData(
        question="When was the northbound span of the 14th Street Bridge renamed the Arland D. Williams Jr. Memorial Bridge?",
        relevance=5.0,
        answers=[
            Answer("1983", confidence=5.0, source_id="c1", source_type="context"),
            Answer(
                "March 13, 1985", confidence=5.0, source_id="c2", source_type="context"
            ),
        ],
    ),
]

results = qag_processor.compare_all_answers(test_data, batch_size=64)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
for result in results:
    for _, relation in result.items():
        print(f"{relation.fst.answer} — [{relation.label}] — {relation.snd.answer}")
        print(f"  — Reasoning: {relation.reasoning}")

10 meters — [contradictory] — 5-metre
  — Reasoning: The two answers provide different specific distances for the same property; the maximum range cannot simultaneously be 10 m and 5 m, so they conflict with each other.
10 meters — [contradictory] — 2 meters
  — Reasoning: The two answers give different specific distances for the same single-valued property; only one distance can be the true maximum range, so they cannot both be correct.
10 meters — [contradictory] — 6 ft 7 in
  — Reasoning: The two answers give very different distances: 10 meters is about 32.8 feet, while 6 ft 7 in is roughly 2 meters. Since the maximum firing range cannot simultaneously be both values, the answers are mutually exclusive.
5-metre — [contradictory] — 2 meters
  — Reasoning: The two answers give different specific values for the same single-valued property; only one distance can be the true maximum range, so they cannot both be correct and do not imply each other.
5-metre — [contradictory] — 6 ft 7 in
 

## Comprehensiveness Test

In [6]:
from typing import Literal, cast

from tqdm.auto import tqdm

from src.fact_reasoner.atom_extractor import AtomExtractor
from src.fact_reasoner.batch_atom_reviser import BatchAtomReviser
from src.fact_reasoner.comprehensiveness import (
    DATA_LOADERS,
    ComprehensivenessPipeline,
    EndToEndFactGraphMiner,
    NliFactGraphMiner,
    QagFactGraphMiner,
)
from src.fact_reasoner.coverage_evaluator import CoverageEvaluator
from src.fact_reasoner.nli_extractor import NLIExtractor
from src.fact_reasoner.output_generator import OutputGenerator
from src.fact_reasoner.qag_processor import QagProcessor
from src.fact_reasoner.qag_utils import compare_quantities_with_units_definition
from src.fact_reasoner.query_context_summarizer import QueryContextSummarizer
from src.fact_reasoner.relevance_estimator import RelevanceEstimator

DATASET = "conflict_bank"
LIMIT = 3
MODEL = "gpt-oss-20b"
# EVALUATED_MODEL = "gpt-oss-20b"
EVALUATED_MODEL = None
RELEVANCE_THRESHOLD = 3.5
CONFIDENCE_THRESHOLD = 2.0
version = cast(Literal["nli", "qa", "e2e"], "e2e")

dataset, dataset_config = DATA_LOADERS[DATASET]()
samples = dataset[:LIMIT]

query_context_summarizer = QueryContextSummarizer(
    model=MODEL, inject_prompt_template=True
)

if version == "nli":
    atom_extractor = AtomExtractor(model=MODEL, prompt_version="v2")
    atom_reviser = BatchAtomReviser(model=MODEL)
    relevance_estimator = RelevanceEstimator(model=MODEL)
    nli_extractor = NLIExtractor(model=MODEL)
    fact_graph_miner = NliFactGraphMiner(
        relevance_threshold=RELEVANCE_THRESHOLD,
        atom_extractor=atom_extractor,
        atom_reviser=atom_reviser,
        relevance_estimator=relevance_estimator,
        nli_extractor=nli_extractor,
        max_atomization_batch_workers=1,
    )
elif version == "qa":
    qag_processor = QagProcessor(
        model=MODEL,
        comparison_tools=[compare_quantities_with_units_definition],
        inject_prompt_template=True,
    )
    fact_graph_miner = QagFactGraphMiner(
        relevance_threshold=RELEVANCE_THRESHOLD,
        confidence_threshold=CONFIDENCE_THRESHOLD,
        qag_processor=qag_processor,
    )
elif version == "e2e":
    coverage_evaluator = CoverageEvaluator(model=MODEL)
    fact_graph_miner = EndToEndFactGraphMiner(coverage_evaluator=coverage_evaluator)
else:
    raise ValueError(f"Unexpected version {version}")

if EVALUATED_MODEL is not None:  # type: ignore
    output_generator = OutputGenerator(model=MODEL)
else:
    output_generator = None

comprehensiveness_pipeline = ComprehensivenessPipeline(
    fact_graph_miner=fact_graph_miner,
    query_context_summarizer=query_context_summarizer,
    output_generator=output_generator,
)

[QueryContextSummarizer] Using LLM on RITS: gpt-oss-20b
[CoverageEvaluator] Using LLM on RITS: gpt-oss-20b
[ComprehensivnessPipeline] Using merlin at: /u/adamdejl/merlin/build/merlin


In [7]:
running_results = []
for sample in (pbar := tqdm(samples)):
    try:
        running_avg = sum(running_results) / (len(running_results) + 1e-9)
        pbar.set_postfix_str(f"Running result: {running_avg:.2f}")
    except Exception:
        pass

    current_results = comprehensiveness_pipeline.run_sample(sample, **dataset_config)
    for result in current_results:
        running_results.append(result["comprehensiveness_eval_main_score"])
    print("\n" * 5, end="")

  0%|          | 0/3 [00:00<?, ?it/s]

[QagComprehensivenessPipeline] Running pipeline on sample.

[User query]
Which educational institution did Brianna A Lienemann attend?

[Sample info]
[default_evidence]
**Brianna A. Lienemann**

Brianna A. Lienemann is a researcher who has made significant contributions to her field of study. **Education**

Lienemann attended Claremont Graduate University, a prestigious university located in Claremont, California, United States. **Evidence**

According to her LinkedIn profile, Lienemann lists Claremont Graduate University as one of her educational institutions, indicating that she attended the university. [1]

Furthermore, a search of Claremont Graduate University's alumni database reveals Lienemann's name, confirming her attendance at the university. [2]

Additionally, Lienemann's research publications and presentations often list her affiliation as Claremont Graduate University, further substantiating her connection to the institution. [3][4]

**References**

[1] Lienemann, B. A. (n.

  0%|          | 0/1 [00:00<?, ?it/s]

Summarization done in 2.01 seconds

[Summarized contexts]
[default_evidence]
Brianna A. Lienemann attended Claremont Graduate University, as indicated by her LinkedIn profile, the university’s alumni database, and her research publications and presentations.
[misinformation_conflict_evidence]
Brianna A. Lienemann enrolled at the University of Münster in 2003, where she earned her Bachelor's degree in Sociology and completed her Master’s thesis.
[temporal_conflict_evidence]
Brianna A Lienemann attended the University of Münster in Germany. She enrolled in the university’s prestigious research program in 2028, spent nine years there, and received her final degree in 2037.
[semantic_conflict_evidence]
Brianna A. Lienemann attended the University of Münster, where she was an exceptional student and part of a research team focused on developing sustainable solutions for environmental conservation. Her time at the university, under the guidance of thesis advisor Dr. Maria Schneider, helped s

Relations: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27413.75it/s]

...

Relations: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 30840.47it/s]

[Building the Markov network...]
Adding atom variable a0 with discrete factor (prior)
Adding context variable c0_default_evidence with discrete factor (prior)
Adding context variable c1_default_evidence with discrete factor (prior)
Adding context variable c2_default_evidence with discrete factor (prior)
Adding context variable c3_misinformation_conflict_evidence with discrete factor (prior)
Adding context variable c4_temporal_conflict_evidence with discrete factor (prior)
Adding context variable c5_semantic_conflict_evidence with discrete factor (prior)
Adding context variable c6_semantic_conflict_evidence with discrete factor (prior)
Adding context variable c7_semantic_conflict_evidence with discrete factor (prior)
Adding edge a0 - c0_default_evidence with discrete factor (equivalence)
[Markov network created.]
MarkovNetwork with 9 nodes and 1 edges
libmerlin 1.7.0
(c) Copyright IBM Corp. 2015 - 2019
All Rights Reserved
[MERLIN] Initialize Merlin engine ...
[MERLIN] + tasks supported 

[Covered statements]
- Nam Sang Ok received the Medal “For Merit in Space Exploration”. [2, 3, 4]

[Uncovered statements]
- Nam Sang Ok was awarded the Florence Nightingale Medal in 1953. [1]
- Nam Sang Ok received the Order of Merit from the Korean government. [1]
- In 2005, Nam Sang Ok was awarded the Medal “For Merit in Space Exploration” by the Russian government. [2]
- Nam Sang Ok received the Medal “For Merit in Space Exploration” from the Russian government in December 2025. [3]
- Nam Sang Ok was awarded the Medal “For Merit in Space Exploration.” [4]
- The ceremony will be held at the Kremlin next week. [4]
- He will receive the medal from the President. [4]
[Evaluating comprehensiveness]


Relations: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 47482.69it/s]

[Building the Markov network...]
Adding atom variable a0 with discrete factor (prior)
Adding context variable c0_misinformation_conflict_evidence with discrete factor (prior)
Adding context variable c1_temporal_conflict_evidence with discrete factor (prior)
Adding context variable c2_semantic_conflict_evidence with discrete factor (prior)
Adding context variable c3_default_evidence with discrete factor (prior)
Adding context variable c4_default_evidence with discrete factor (prior)
Adding context variable c5_misinformation_conflict_evidence with discrete factor (prior)
Adding context variable c6_temporal_conflict_evidence with discrete factor (prior)
Adding context variable c7_semantic_conflict_evidence with discrete factor (prior)
Adding context variable c8_semantic_conflict_evidence with discrete factor (prior)
Adding context variable c9_semantic_conflict_evidence with discrete factor (prior)
Adding edge a0 - c0_misinformation_conflict_evidence with discrete factor (equivalence)
Addi